# LSTM 做词性预测
前面我们讲了词嵌入以及 n-gram 模型做单词预测，但是目前还没有用到 RNN，在最后这一次课中，我们会结合前面讲的所有预备知识，教大家如何使用 LSTM 来做词性预测。

## 模型介绍
对于一个单词，会有这不同的词性，首先能够根据一个单词的后缀来初步判断，比如 -ly 这种后缀，很大概率是一个副词，除此之外，一个相同的单词可以表示两种不同的词性，比如 book 既可以表示名词，也可以表示动词，所以到底这个词是什么词性需要结合前后文来具体判断。

根据这个问题，我们可以使用 lstm 模型来进行预测，首先对于一个单词，可以将其看作一个序列，比如 apple 是由 a p p l e 这 5 个单词构成，这就形成了 5 的序列，我们可以对这些字符构建词嵌入，然后输入 lstm，就像 lstm 做图像分类一样，只取最后一个输出作为预测结果，整个单词的字符串能够形成一种记忆的特性，帮助我们更好的预测词性。

![](https://ws3.sinaimg.cn/large/006tKfTcgy1fmxi67w0f7j30ap05qq2u.jpg)

接着我们把这个单词和其前面几个单词构成序列，可以对这些单词构建新的词嵌入，最后输出结果是单词的词性，也就是根据前面几个词的信息对这个词的词性进行分类。

下面我们用例子来简单的说明

In [1]:
import torch
from torch import nn
from torch.autograd import Variable

In [2]:
train_data=[("The dog ate the apple".split(),["DET","NN","V","DET","NN"]),
            ("Everybody read that book".split(),["NN","V","DET","NN"])]

In [3]:
train_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

In [5]:
word_to_idx={}
tag_to_idx={}
for context,tag in train_data:
    for word in context:
        if word.lower() not in word_to_idx:
            word_to_idx[word.lower()]=len(word_to_idx)
    for label in tag:
        if label.lower() not in tag_to_idx:
            tag_to_idx[label.lower()]=len(tag_to_idx)

In [7]:
print(word_to_idx,'\n',tag_to_idx)

{'the': 0, 'dog': 1, 'ate': 2, 'apple': 3, 'everybody': 4, 'read': 5, 'that': 6, 'book': 7} 
 {'det': 0, 'nn': 1, 'v': 2}


In [9]:
#然后我们对字母进行编码
alphabet = 'abcdefghijklmnopqrstuvwxyz'
char_to_idx={}
for i in range(len(alphabet)):
    char_to_idx[alphabet[i]]=i

In [10]:
print(char_to_idx)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [11]:
#接着我们可以构建训练数据
def make_sequence(x,dic):
    idx=[dic[i.lower()] for i in x]
    idx=torch.LongTensor(idx)
    return idx

In [12]:
make_sequence('apple',char_to_idx)#构建char level的数据集

tensor([ 0, 15, 15, 11,  4])

In [16]:
train_data[0][0]

['The', 'dog', 'ate', 'the', 'apple']

In [17]:
make_sequence(train_data[0][0],word_to_idx)#构建word level的数据集

tensor([0, 1, 2, 0, 3])

In [15]:
#构建char level的数据集
class char_lstm(nn.Module):
    def __init__(self,n_char,char_dim,char_hidden):
        super(char_lstm,self).__init__()
        # 构造char的embedding table
        self.char_embed=nn.Embedding(n_char,char_dim)
        self.lstm=nn.LSTM(char_dim,char_hidden)#input_size,output_size,num_layers
    def forward(self,x):
        x=self.char_embed(x)#[seq,1]
        out,_=self.lstm(x)
        return out[-1]#[batch,hidden]

In [57]:
#构建词性分类的lstm模型
class lstm_tagger(nn.Module):
    def __init__(self,n_word,n_char,char_dim,word_dim,char_hidden,word_hidden,n_tag):
        super(lstm_tagger,self).__init__()
        self.word_embed=nn.Embedding(n_word,word_dim)
        self.char_lstm=char_lstm(n_char,char_dim,char_hidden)#放入自定义的模型,学到了.
        self.word_lstm=nn.LSTM(word_dim+char_hidden,word_hidden)#char和word的向量先级联一下,然后送入lstm
        self.classify=nn.Linear(word_hidden,n_tag)
    def forward(self,x,word):# x=tensor([[0, 1, 2, 0, 3]]),word=['The', 'dog', 'ate', 'the', 'apple']
        char=[]
        for w in word:
            char_list=make_sequence(w,char_to_idx)# tensor([ 0, 15, 15, 11,  4])
            char_list=char_list.unsqueeze(1)# (seq,1) 满足 lstm 输入条件
            char_infor=self.char_lstm(Variable(char_list))#(batch,char_hidden)
            char.append(char_infor)
        char=torch.stack(char,dim=0)#(seq,batch,char_dim)#在第一个维度进行合并.char是个list,其中元素是tensor

        x=self.word_embed(x)
        x=x.permute(1,0,2)#(seq,batch,word_dim)
        # 沿着特征通道将每个词的词嵌入和字符 lstm 输出的结果拼接在一起
        x=torch.cat((x,char),dim=2)#(seq,batch,word_dim+char_dim)
        x,_=self.word_lstm(x)

        s,b,h=x.shape
        x=x.view(-1,h)
        out=self.classify(x)#[seq,n_tag]

        return out

In [60]:
net = lstm_tagger(n_word=len(word_to_idx),
                  n_char=len(char_to_idx),
                  char_dim=10,
                  word_dim=100,
                  char_hidden=50,
                  word_hidden=128,
                  n_tag=len(tag_to_idx))
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters(),lr=0.01)

# 开始训练
for e in range(300):
    train_loss = 0
    for word, tag in train_data:
        word_list = make_sequence(word, word_to_idx).unsqueeze(0) # 添加第一维 batch
        tag = make_sequence(tag, tag_to_idx)
        word_list = Variable(word_list)
        tag = Variable(tag)
        # 前向传播
        out = net(word_list, word)
        loss = criterion(out, tag)
        train_loss += loss.item()
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (e + 1) % 50 == 0:
        print('Epoch: {}, Loss: {:.5f}'.format(e + 1, train_loss / len(train_data)))

Epoch: 50, Loss: 0.89621
Epoch: 100, Loss: 0.71674
Epoch: 150, Loss: 0.53297
Epoch: 200, Loss: 0.37616
Epoch: 250, Loss: 0.26186
Epoch: 300, Loss: 0.18562


### 扩展:nn.CrossEntropyLoss(output, label):
tensor1可以是未经softmax的概率分布,如词性预测中是(seq,n_tag)

label可以是未经one_hot化的tensor,如此处是真实index,跟tensorflow中相似.

In [54]:
a=torch.Tensor([[ 0.0059,  0.0458,  0.0190]])#不经过softmax都可以,注意一下
b=torch.LongTensor([0])#未经one_hot都可以
print(criterion(a,b))

tensor(1.1164)


In [61]:
#看一下预测结果
net=net.eval()
test_sent='Everybody ate the apple'
test=make_sequence(test_sent.split(),word_to_idx).unsqueeze(0)
out=net(Variable(test),test_sent.split())

In [63]:
print(out)
_,out_to_idx=out.max(dim=-1)
print(out_to_idx)

tensor([[-1.2641,  1.6261, -0.3703],
        [-0.6188, -0.1378,  1.1287],
        [ 1.8942, -0.7132, -0.7472],
        [-0.4547,  1.7929, -1.1326]], grad_fn=<ThAddmmBackward>)
tensor([1, 2, 0, 1])


In [64]:
print(tag_to_idx)

{'det': 0, 'nn': 1, 'v': 2}


最后可以得到上面的结果，因为最后一层的线性层没有使用 softmax，所以数值不太像一个概率，但是每一行数值最大的就表示属于该类，可以看到第一个单词 'Everybody' 属于 nn，第二个单词 'ate' 属于 v，第三个单词 'the' 属于det，第四个单词 'apple' 属于 nn，所以得到的这个预测结果是正确的

### 小结:
#### 一.输入输出的shape变化
词性预测相当于在sequence维度上,每个单词都进行词性标注,而不是对某一个单词进行预测.

比如说输入input shape=(batch,sequence),label shape=(batch,sequence)

我们要让模型的输出output shape=(batch,sequence,n_tag),然后tensor.max(dim=-1)即是输出

#### 二.模型解释
本模型从char level和word level都做了处理,然后最后做了torch.cat处理.此处就体现了torch的优越性,因为是动态的,所以可以在模型中用for循环,比tensorflow要方便.

以'apple'为例:不仅word层面进行了LSTM Cell的计算,还把他拆解为'a','p','p','l','e',做完LSTM序列计算后取最后一个time step的输出,维度还是[1,char_dim]而不是[5,char_dim]

#### 三.torch.stack(list)
list是包含很多tensor的列表,假设len(list)=5,list中的每个tensor shape=(2,3),然后我们torch.stack(list,dim=)生成一个多1维度的Tensor,如果dim=0,那生成的tensor就是[5,2,3],如果dim=1,tensor就是[2,5,3].

#### 四.注意nn.LSTM()生成包括两部分,一部分是out序列,另一部分是(h,c)不要忘了.
如果只需要out,那就如下:`out,_=self.lstm(input)`

In [92]:
#自己实现一下
class myModel(nn.Module):
    def __init__(self,word_dim,n_word,char_dim,n_char,word_hidden,char_hidden,n_tag):
        super(myModel,self).__init__()
        self.word_embed=nn.Embedding(n_word,word_dim)
        self.char_lstm=char_lstm(n_char,char_dim,char_hidden)
        self.word_lstm=nn.LSTM(word_dim+char_hidden,word_hidden)
        self.classify=nn.Linear(word_hidden,n_tag)
    def forward(self,x,words):
        x=self.word_embed(x)#[1,seq]-->[1,seq,word_dim]
        charTensor=[]
        for word in words:
            #将word展开为char序列
            chars=make_sequence(word,char_to_idx)#[char_num]
            chars=chars.unsqueeze(1)#[char_num,1]
            
            #送入char的LSTM模型
            chars_tensor=self.char_lstm(chars)#[1,char_hidden]
            charTensor.append(chars_tensor)#seq个[1,char_hidden]组成的list
        #将list stack为tensor
        charTensor=torch.stack(charTensor,dim=1)#看一下stack的计算过程,[1,seq,char_hidden]
#         print(charTensor.shape)
        #拼接,permute
        x=torch.cat((x,charTensor),dim=-1)#[1,seq,word_dim+char_hidden]
#         print(x.shape)
        x=x.permute(1,0,2)#[seq,1,word_dim+char_hidden]
        #进入word LSTM
        x,_=self.word_lstm(x)#[seq,1,word_hidden],注意!此处的word_lstm不仅返回out,还返回(h,c)
        #reshape
        seq,batch,hidden=x.shape
        x=x.view(batch*seq,-1)#[seq*1,word_hidden]
        #classify
        out=self.classify(x)#[seq,n_tag]
        return out  

In [93]:
memodel = myModel(n_word=len(word_to_idx),
                  n_char=len(char_to_idx),
                  char_dim=10,
                  word_dim=100,
                  char_hidden=50,
                  word_hidden=128,
                  n_tag=len(tag_to_idx))
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(memodel.parameters(),lr=0.01)

# 开始训练
for e in range(300):
    train_loss = 0
    for word, tag in train_data:
        word_list = make_sequence(word, word_to_idx).unsqueeze(0) # 添加第一维 batch
        tag = make_sequence(tag, tag_to_idx)
        word_list = Variable(word_list)
        tag = Variable(tag)
        # 前向传播
        out = memodel(word_list, word)
        loss = criterion(out, tag)
        train_loss += loss.item()
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (e + 1) % 50 == 0:
        print('Epoch: {}, Loss: {:.5f}'.format(e + 1, train_loss / len(train_data)))

Epoch: 50, Loss: 0.90201
Epoch: 100, Loss: 0.69583
Epoch: 150, Loss: 0.49335
Epoch: 200, Loss: 0.33046
Epoch: 250, Loss: 0.22147
Epoch: 300, Loss: 0.15416
